In [1]:
# SecureString parameter stored in AWS System Manager that holds a GitHub personal access token.
parameter_name = 'github_token'

# Organization names that contain members considered employees.
organization_names = ['aws', 'awslabs', 'aws-amplify', 'aws-samples']

# Repositories for which reports will be generated.
repo_names = ['aws-amplify/amplify-android',
              'aws-amplify/aws-sdk-android',
              'awslabs/aws-mobile-appsync-sdk-android']

In [2]:
# Retrieve the GitHub token from SSM to prevent oops-I-pushed-credentials-to-GitHub uh-ohs.

import boto3
from github import Github

ssm = boto3.client('ssm')
response = ssm.get_parameter(Name=parameter_name, WithDecryption=True)
token = response['Parameter']['Value']
github = Github(token)

In [3]:
# Build a set of GitHub handles of employees to later exclude their activity from the report.
# TODO: Is this kosher? Perhaps TAMs, SAs, etc are opening issues on behalf of their customers?

employees = set()

for organization_name in organization_names:
    organization = github.get_organization(organization_name)
    
    for member in organization.get_members():
        employees.add(member.login)

In [4]:
# Loop through each repository, grab all issues, and create a DataFrame for each.

import pandas as pd

repos = {}

for repo_name in repo_names:
    repo = github.get_repo(repo_name)
    issues = []

    for issue in repo.get_issues(state='all'):
        labels = pd.array([label.name for label in issue.labels])
        
        if issue.pull_request is None and issue.user.login not in employees:
            issues.append([labels, issue.created_at, issue.closed_at])
        
    repos[repo_name] = pd.DataFrame(issues, columns=['labels', 'created_at', 'closed_at'])

In [14]:
# Using the DataFrames, process and grab the counts of issues. Go through each label and gets counts for each label and other applied labels.

from datetime import datetime
from IPython.display import display, HTML

seven_days_ago = datetime.now() - pd.Timedelta('7 days')
thirty_days_ago = datetime.now() - pd.Timedelta('30 days')

open_counts = []
label_counts = []

for repo_name in repo_names:
    r = repos[repo_name]
    labels = {label for labels in repos[repo_name].labels for label in labels}

    open_issues = r[~(r.closed_at > '1970-01-01')]
    last_week_open_issues = r[(r.created_at < seven_days_ago) & ~(r.closed_at < seven_days_ago)]
    last_month_open_issues = r[(r.created_at < thirty_days_ago) & ~(r.closed_at < thirty_days_ago)]
    open_issues_count = len(open_issues.index)
    last_week_open_issues_count = len(last_week_open_issues.index)
    last_month_open_issues_count = len(last_month_open_issues.index)
    
    open_counts.append([repo_name,
                        open_issues_count,
                        last_week_open_issues_count,
                        open_issues_count - last_week_open_issues_count,
                        last_month_open_issues_count,
                        open_issues_count - last_month_open_issues_count])
    
    for label in sorted(labels):
        label_mask = open_issues.labels.apply(lambda l: label in l)
        issues = open_issues[label_mask]
        last_week_label_issues = issues[(issues.created_at < seven_days_ago) & ~(issues.closed_at < seven_days_ago)]
        last_month_label_issues = issues[(issues.created_at < thirty_days_ago) & ~(issues.closed_at < thirty_days_ago)]
        label_issues_count = len(issues.index)
        last_week_label_issues_count = len(last_week_label_issues.index)
        last_month_label_issues_count = len(last_month_label_issues.index)
        
        if label_issues_count or last_week_label_issues_count or last_month_label_issues_count:
            label_counts.append([repo_name,
                                 label,
                                 '',
                                 label_issues_count,
                                 last_week_label_issues_count,
                                 label_issues_count - last_week_label_issues_count,
                                 last_month_label_issues_count,
                                 label_issues_count - last_month_label_issues_count])

            for other_label in labels - {label}:
                other_label_mask = issues.labels.apply(lambda l: other_label in l)
                other_label_issues = issues[other_label_mask]

                if not other_label_issues.empty:
                    last_week_other_label_issues = other_label_issues[(other_label_issues.created_at < seven_days_ago) & ~(other_label_issues.closed_at < seven_days_ago)]
                    last_month_other_label_issues = other_label_issues[(other_label_issues.created_at < thirty_days_ago) & ~(other_label_issues.closed_at < thirty_days_ago)]
                    other_label_issues_count = len(other_label_issues.index)
                    last_week_other_label_issues_count = len(last_week_other_label_issues.index)
                    last_month_other_label_issues_count = len(last_month_other_label_issues.index)

                    if other_label_issues_count or last_week_other_label_issues_count or last_month_other_label_issues_count:
                        label_counts.append([repo_name,
                                             label,
                                             other_label,
                                             other_label_issues_count,
                                             last_week_other_label_issues_count,
                                             other_label_issues_count - last_week_other_label_issues_count,
                                             last_month_other_label_issues_count,
                                             other_label_issues_count - last_month_other_label_issues_count])
                
open_df = pd.DataFrame(open_counts, columns=['Repo', 'Open', 'Last Week', 'WoW', 'Last Month', 'MoM'])
labels_df = pd.DataFrame(label_counts, columns=['Repo', 'Label', 'Other Label', 'Open', 'Last Week', 'WoW', 'Last Month', 'MoM'])

In [15]:
# Build the report for display.

pd.set_option('display.max_rows', 500)

display(HTML('<h1>Open Source – Ops Report</h1>'))
display(HTML(f'<em>Generated on {datetime.now()}'))

display(HTML('<h2>Open Issues</h2>'))
display(open_df)

display(HTML('<h2>Open Issues by Label</h2>'))

for repo_name in repo_names:
    display(HTML(f'<h3>{repo_name}</h3>'))
    display(labels_df[labels_df.Repo == repo_name])

,Repo,Open,Last Week,WoW,Last Month,MoM
0,aws-amplify/amplify-android,9,9,0,7,2
1,aws-amplify/aws-sdk-android,126,128,-2,121,5
2,awslabs/aws-mobile-appsync-sdk-android,42,45,-3,43,-1


,Repo,Label,Other Label,Open,Last Week,WoW,Last Month,MoM
0,aws-amplify/amplify-android,API,,3,3,0,3,0
1,aws-amplify/amplify-android,API,Bug,1,1,0,1,0
2,aws-amplify/amplify-android,API,Clarification Needed,1,1,0,1,0
3,aws-amplify/amplify-android,API,Feature Request,1,1,0,1,0
4,aws-amplify/amplify-android,API,Core,1,1,0,1,0
5,aws-amplify/amplify-android,Bug,,3,3,0,3,0
6,aws-amplify/amplify-android,Bug,API,1,1,0,1,0
7,aws-amplify/amplify-android,Bug,DataStore,2,2,0,2,0
8,aws-amplify/amplify-android,Bug,Clarification Needed,1,1,0,1,0
9,aws-amplify/amplify-android,Clarification Needed,,2,2,0,2,0


,Repo,Label,Other Label,Open,Last Week,WoW,Last Month,MoM
27,aws-amplify/aws-sdk-android,APIGateway,,4,4,0,4,0
28,aws-amplify/aws-sdk-android,APIGateway,Feature Request,1,1,0,1,0
29,aws-amplify/aws-sdk-android,APIGateway,Usage Question,3,3,0,3,0
30,aws-amplify/aws-sdk-android,AWSMobileClient,,34,33,1,30,4
31,aws-amplify/aws-sdk-android,AWSMobileClient,IoT,1,1,0,1,0
32,aws-amplify/aws-sdk-android,AWSMobileClient,Needs Info from Requester,6,5,1,4,2
33,aws-amplify/aws-sdk-android,AWSMobileClient,Core,1,1,0,1,0
34,aws-amplify/aws-sdk-android,AWSMobileClient,Documentation,2,2,0,2,0
35,aws-amplify/aws-sdk-android,AWSMobileClient,closing-soon-if-no-response,1,1,0,0,1
36,aws-amplify/aws-sdk-android,AWSMobileClient,Service,1,1,0,1,0


,Repo,Label,Other Label,Open,Last Week,WoW,Last Month,MoM
172,awslabs/aws-mobile-appsync-sdk-android,AppSync,,39,39,0,38,1
173,awslabs/aws-mobile-appsync-sdk-android,AppSync,Documentation,2,2,0,2,0
174,awslabs/aws-mobile-appsync-sdk-android,AppSync,Requesting Feedback,2,2,0,2,0
175,awslabs/aws-mobile-appsync-sdk-android,AppSync,Codegen,4,4,0,4,0
176,awslabs/aws-mobile-appsync-sdk-android,AppSync,Bug,16,16,0,16,0
177,awslabs/aws-mobile-appsync-sdk-android,AppSync,Infrastructure,1,1,0,1,0
178,awslabs/aws-mobile-appsync-sdk-android,AppSync,Pending,1,1,0,1,0
179,awslabs/aws-mobile-appsync-sdk-android,AppSync,Feature Request,13,13,0,13,0
180,awslabs/aws-mobile-appsync-sdk-android,AppSync,Question,8,8,0,7,1
181,awslabs/aws-mobile-appsync-sdk-android,AppSync,Product Review,2,2,0,2,0
